Nearest-Neighbor Interpolation:

This is a simple algorithm where each output pixel is assigned the value of the closest pixel from the input image.
Bilinear Interpolation:

Bilinear interpolation uses the values of the four nearest pixels to estimate the value of the new pixel. It linearly interpolates in both directions to produce smooth transitions.
Normalized SSD:

Sum of Squared Differences (SSD) computes the squared difference between corresponding pixel values. We normalize it by dividing by the total number of pixels in the image to get a comparable measure across different images.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function for Nearest-Neighbor Interpolation
def nearest_neighbor_interpolation(image, scale_factor):
    h, w = image.shape[:2]
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    zoomed_image = np.zeros((new_h, new_w, 3), dtype=np.uint8)
    
    for i in range(new_h):
        for j in range(new_w):
            # Find the nearest pixel from the original image
            nearest_x = int(i / scale_factor)
            nearest_y = int(j / scale_factor)
            zoomed_image[i, j] = image[nearest_x, nearest_y]
    
    return zoomed_image

# Function for Bilinear Interpolation
def bilinear_interpolation(image, scale_factor):
    h, w = image.shape[:2]
    new_h, new_w = int(h * scale_factor), int(w * scale_factor)
    zoomed_image = np.zeros((new_h, new_w, 3), dtype=np.uint8)
    
    for i in range(new_h):
        for j in range(new_w):
            x = i / scale_factor
            y = j / scale_factor
            
            x0 = int(np.floor(x))
            x1 = min(x0 + 1, h - 1)
            y0 = int(np.floor(y))
            y1 = min(y0 + 1, w - 1)
            
            # Interpolation in the x direction
            Ia = image[x0, y0]
            Ib = image[x1, y0]
            Ic = image[x0, y1]
            Id = image[x1, y1]
            
            # Perform bilinear interpolation
            wa = (x1 - x) * (y1 - y)
            wb = (x - x0) * (y1 - y)
            wc = (x1 - x) * (y - y0)
            wd = (x - x0) * (y - y0)
            
            zoomed_image[i, j] = wa * Ia + wb * Ib + wc * Ic + wd * Id
    
    return zoomed_image

# Function to calculate Normalized Sum of Squared Difference (SSD)
def calculate_ssd(original, zoomed):
    diff = original.astype(np.float32) - zoomed.astype(np.float32)
    ssd = np.sum(diff ** 2)
    normalized_ssd = ssd / original.size
    return normalized_ssd

# Load the images (add your paths for the original and zoomed-out images)
original_image = cv2.imread('utils/q05/taylor.jpg')
zoomed_out_image = cv2.imread('utils/q05/taylor_very_small.jpg')

# Zoom back the zoomed-out image by a factor of 4 using Nearest-Neighbor
zoomed_nearest_neighbor = nearest_neighbor_interpolation(zoomed_out_image, 4)

# Zoom back the zoomed-out image by a factor of 4 using Bilinear Interpolation
zoomed_bilinear = bilinear_interpolation(zoomed_out_image, 4)

# Calculate SSD for both methods
ssd_nearest_neighbor = calculate_ssd(original_image, zoomed_nearest_neighbor)
ssd_bilinear = calculate_ssd(original_image, zoomed_bilinear)

# Display the results
print(f"Normalized SSD for Nearest-Neighbor: {ssd_nearest_neighbor}")
print(f"Normalized SSD for Bilinear Interpolation: {ssd_bilinear}")

# Display the images
plt.figure(figsize=(10,5))

plt.subplot(1,3,1)
plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(cv2.cvtColor(zoomed_nearest_neighbor, cv2.COLOR_BGR2RGB))
plt.title("Nearest-Neighbor Zoomed")

plt.subplot(1,3,3)
plt.imshow(cv2.cvtColor(zoomed_bilinear, cv2.COLOR_BGR2RGB))
plt.title("Bilinear Zoomed")

plt.show()

ValueError: operands could not be broadcast together with shapes (560,1000,3) (112,200,3) 